# 컴퓨팅 대상 사용

앞에서 Azure Machine Learning SDK를 사용해 여러 실험을 실행했습니다. 모든 실험은 로컬 컴퓨팅(여기서는 Azure Machine Learning Notebook VM)에서 실행되었습니다. 이번에는 클라우드 컴퓨팅을 활용하여 컴퓨팅 컨텍스트의 확장성을 높일 수 있는 방법을 살펴보겠습니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험용 데이터 준비

이 랩에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀을 실행하여 이 데이터 세트를 만듭니다. 이전 랩에서 해당 데이터 세트를 만든 경우 코드를 실행하면 새 버전이 작성됩니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                        target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                        overwrite=True, # 이름이 같은 기존 파일 바꾸기
                        show_progress=True)

    # 데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 테이블 형식 데이터 세트 등록
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 컴퓨팅 대상 만들기

대부분의 경우 로컬 컴퓨팅 리소스만으로는 대량의 데이터를 처리해야 하는 복잡한 실험이나 장기 실행 실험을 처리하기 어려울 수 있습니다. 이러한 상황에서는 클라우드에서 컴퓨팅 리소스를 동적으로 만들고 사용하는 기능을 활용할 수 있습니다.

Azure ML은 광범위한 컴퓨팅 대상을 지원합니다. 이러한 컴퓨팅 대상은 작업 영역에서 정의하고 실험을 실행하는 데 사용할 수 있으며, 리소스 사용 시에만 비용을 지불하면 됩니다. 첫 번째 연습에서 작업 영역을 설정할 때 만든 클러스터가 있는지 확인하고, 없으면 학습 실험을 실행하는 데 사용할 수 있도록 해당 클러스터를 만듭니다.

> **중요**: *your-compute-cluster*를 실행하기 전에 아래 코드에서 사용자의 컴퓨팅 클러스터 이름으로 변경하십시오.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster" # change to your compute cluster name

# 클러스터가 있는지 확인
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 클러스터가 없으면 생성
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

## 원격 컴퓨팅에서 실험 실행

이제 앞에서 만든 컴퓨팅을 사용하여 실험을 실행할 수 있습니다. 다음 코드는 실험 파일용 폴더를 만듭니다. 이 폴더를 이전 랩에서 이미 만들었더라도 코드를 실행하세요.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

다음으로는 실험용 Python 스크립트 파일을 만듭니다. 그러면 이전 랩에서 사용한 스크립트를 덮어쓰게 됩니다.

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

이제 작성된 컴퓨팅에서 실험을 실행할 수 있습니다. 

> **참고**: 이번에는 실험을 실행하는 데 시간이 훨씬 오래 걸립니다. conda 환경이 포함된 이미지를 작성한 다음 클러스터 노드를 시작하고 이미지를 배포해야 스크립트를 실행할 수 있기 때문입니다. 따라서 당뇨병 학습 스크립트와 같은 간단한 실험에서는 효율적이지 않을 수도 있습니다. 하지만 몇 시간이 걸리는 훨씬 복잡한 실험을 실행해야 하는 경우 확장성이 더 높은 컴퓨팅을 동적으로 만들면 전체 실험 시간을 크게 줄일 수도 있습니다.

In [ ]:
from azureml.core import Environment, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# 실험용 Python 환경 만들기
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Azure ML의 종속성 관리 허용
diabetes_env.docker.enabled = True # Docker 컨테이너 사용

# 패키지 종속성 집합 만들기
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                                             pip_packages=['azureml-sdk','pyarrow'])

# 환경에 종속성 추가
diabetes_env.python.conda_dependencies = diabetes_packages

# 환경 등록(이전 랩을 완료하지 않은 경우에 한함)
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 추정기 만들기
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')],
              compute_target = cluster_name, # 이전에 만든 컴퓨팅 대상 사용
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# 실험 만들기
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()

실험이 실행되는 동안 위의 위젯이나 [Azure Machine Learning Studio](https://ml.azure.com)에서 컴퓨팅 상태를 확인할 수 있습니다. 아래 코드를 사용하여 컴퓨팅 상태를 확인할 수도 있습니다.

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

상태가 *steady*에서 *resizing*으로 바뀌려면 시간이 오래 걸리므로 잠시 휴식을 취하셔도 됩니다. 실행이 완료될 때까지 커널을 차단하려면 아래 셀을 실행합니다.

In [ ]:
run.wait_for_completion()

실험이 완료되고 나면 실험 실행에서 생성된 파일과 메트릭을 가져올 수 있습니다. 이번에는 이미지 작성 및 컴퓨팅 관리용 로그가 파일에 포함됩니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이제 실험을 통해 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Azure ML compute'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

>**추가 정보**: Azure Machine Learning의 컴퓨팅 대상에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)를 참조하세요.